In [2]:
cd ../../../../Apps/Python/bolsao-api

C:\Users\luisr\Desktop\Repositories\Apps\Python\bolsao-api


In [3]:
from warnings import filterwarnings as fws; fws('ignore')
import json, pandas as pd
from copy import deepcopy

# ---
# custom modules

from modules.mongo_requests import last_prediction_record
from modules.comando_endpoints import get_clusters_events #, polygons_geojson
from modules.waze_endpoints import get_clusters_waze_partner_alerts
from modules.geojson_conversion import drop_props, add_props, update_props_by_id, geojson_obj

---

## Load local geojson

In [3]:
clusters_geojson = json.loads(open('static/clusters/polygons_micro.geojson').read())

# Extra
clusters_df = pd.read_csv('./static/clusters/clusters_micro.csv')
clusters_json = json.dumps(clusters_df.to_dict(orient='records'), indent=4)

#### Filter geojson features

In [4]:
outlier_polygon = lambda feat: feat['properties']['sublabel'] != -1

clusters_geojson['features'] = list(filter(outlier_polygon, clusters_geojson['features']))

#### Delete geojson properties

In [5]:
keep_cols = [
    'sublabel', 'main_neighborhood', 'main_route', 'main_street_number_range',
    'lat_centroid', 'lng_centroid', 'label_count', 'area_box', 'area_circle'
]

clusters_geojson['features'] = drop_props(clusters_geojson['features'], keep_cols)

## Get external data

#### Waze data

In [6]:
waze = get_clusters_waze_partner_alerts()

#### Comando data

In [7]:
comando = get_clusters_events()

Comando open events: Request attempt (1) 


#### Probability data

In [8]:
prob = pd.DataFrame(last_prediction_record(mongo_timeout=1000*5, load_limit=500))

Endpoint Request: /predict


## Append properties to geojson by id field

#### Append waze data

In [9]:
waze_drop = ['alerts_ids', 'waterbag_alerts_ids']

clusters_geojson['features'] = update_props_by_id(        # for same lenght and order features and dataframes
    clusters_geojson['features'],
    waze,
    id_col='sublabel',
    id_col_df='cluster_id',
    drop=waze_drop
)

#### Append comando data

In [10]:
comando_drop = ['events_ids', 'waterbag_events_ids']

clusters_geojson['features'] = update_props_by_id(        # for same lenght and order features and dataframes
    clusters_geojson['features'],
    comando,
    id_col='sublabel',
    id_col_df='cluster_id',
    drop=comando_drop
)

#### Append probability data

In [11]:
prob_drop = ['_id', 'cluster', 'range']

clusters_geojson['features'] = update_props_by_id(        # for same lenght and order features and dataframes
    clusters_geojson['features'],
    prob,
    id_col='sublabel',
    id_col_df='cluster_id',
    drop=prob_drop
)

---
## Build full dinamic geojson polygons endpoint

In [22]:
# ---
# Local resources

#### Load clusters
clusters_df = pd.read_csv('static/clusters/clusters_micro.csv')
clusters_df = clusters_df[clusters_df['sublabel'] != -1]

# ---
# Clusters endpoint settings

#### Clusters geojson properties settings
geometry_cols = ['lng_min', 'lng_max', 'lat_min', 'lat_max']
keep_cols_clusters = [
    'sublabel', 'main_neighborhood', 'main_route', 'main_street_number_range',
    'lat_centroid', 'lng_centroid', 'label_count', 'area_box', 'area_circle'
]
main_cols_clusters = keep_cols_clusters + geometry_cols
waze_drop = []
comando_drop = []
prob_drop = ['_id', 'cluster', 'range']

# ---
# Dinamic clusters endpoint functions

## Get and append current monitoring data to json
def clusters_status(keep_geometry=False):
    cols = keep_cols_clusters
    if keep_geometry: cols += geometry_cols
    objs = deepcopy(clusters_df[cols]) # Copy features to keep originals intact
    waze = get_clusters_waze_partner_alerts() # Waze data
    comando = get_clusters_events() # Comando data
    prob = pd.DataFrame(last_prediction_record(mongo_timeout=1000*5, load_limit=500)) # Probability data
    for df, drop_cols in zip([waze, comando, prob], [waze_drop, comando_drop, prob_drop]):
        objs = objs.join(df.set_index('cluster_id').drop(drop_cols, axis=1), on='sublabel')
    return objs

## See final result

In [23]:
clusters_now = clusters_status(keep_geometry=True)

clusters_now

Comando open events: Request attempt (1) 
Endpoint Request: /predict


,sublabel,main_neighborhood,main_route,main_street_number_range,lat_centroid,lng_centroid,label_count,area_box,area_circle,lng_min,...,events_status,waterbag_events,waterbag_events_ids,waterbag_events_status,timestamp,date,time,probability,confidence,label
1,0,Barra da Tijuca,Avenida Armando Lombardi,3098 - 67,-23.006631,-43.310232,114,0.047620,0.178174,-43.312829,...,0,0,[],0,2022-12-22 03:20:00.003000,2022-12-22,00:20:00,0.630775,0.261550,1.0
2,1,Catete,Rua do Catete,228 - 139,-22.926423,-43.176842,99,0.057771,0.077165,-43.178604,...,0,0,[],0,2022-12-22 03:20:00.003000,2022-12-22,00:20:00,0.365749,0.268502,0.0
3,2,Copacabana,Rua Tonelero,236 - 9,-22.966793,-43.185999,43,0.040122,0.042480,-43.187500,...,0,0,[],0,2022-12-22 03:20:00.003000,2022-12-22,00:20:00,0.126498,0.747004,0.0
4,3,Ipanema,Avenida Epitácio Pessoa,1910 - 1602,-22.979735,-43.202540,36,0.039586,0.069447,-43.204100,...,0,0,[],0,2022-12-22 03:20:00.003000,2022-12-22,00:20:00,0.051667,0.896666,0.0
5,4,Barra da Tijuca,Avenida Ministro Ivan Lins,1770 - 11,-23.012736,-43.298339,36,0.052712,0.167301,-43.300486,...,0,0,[],0,2022-12-22 03:20:00.003000,2022-12-22,00:20:00,0.819806,0.639613,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,76,Bangu,Rua da Feira,540 - 359,-22.880275,-43.472239,10,0.007294,0.055444,-43.472600,...,0,0,[],0,NaN,NaN,NaN,NaN,NaN,NaN
76,77,Jardim Carioca,Rua Muiatuca,74 - 73,-22.805012,-43.196837,9,0.000534,0.000794,-43.197100,...,0,0,[],0,NaN,NaN,NaN,NaN,NaN,NaN
77,78,Lagoa,Avenida Epitácio Pessoa,3000 - 1015,-22.972562,-43.203594,9,0.024048,0.055909,-43.204600,...,0,0,[],0,NaN,NaN,NaN,NaN,NaN,NaN
78,81,Bangu,Rua Francisco Real,1365 - 974,-22.879638,-43.456768,9,0.046745,0.132317,-43.458700,...,0,0,[],0,NaN,NaN,NaN,NaN,NaN,NaN
